In [270]:
import networkx as nx
import sympy 
from itertools import combinations
from collections import defaultdict, Counter
from networkx.algorithms.dominating import is_dominating_set
import pubchempy as pcp
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from IPython.display import display, Math, HTML
import pandas as pd

In [271]:
class MolecularGraphExtractor:
    def __init__(self, compound_name, relabel=True, visualize=True):
        self.compound_name = compound_name
        self.relabel = relabel
        self.visualize = visualize
        self.smiles = self.get_smiles_from_pubchem()
        self.graph, self.mol = self.smiles_to_nx_graph()

        if self.relabel:
            self.graph = self.relabel_graph_to_integers()

        if self.visualize:
            self.show_all_views()

    def get_smiles_from_pubchem(self):
        compounds = pcp.get_compounds(self.compound_name, namespace='name')
        if not compounds:
            raise ValueError(f"No compound found for name: {self.compound_name}")
        return compounds[0].canonical_smiles

    def smiles_to_nx_graph(self):
        mol = Chem.MolFromSmiles(self.smiles)
        AllChem.Compute2DCoords(mol)
        G = nx.Graph()

        for atom in mol.GetAtoms():
            idx = atom.GetIdx()
            symbol = atom.GetSymbol()
            pos = mol.GetConformer().GetAtomPosition(idx)
            G.add_node(idx, element=symbol, pos=(pos.x, pos.y))

        for bond in mol.GetBonds():
            u, v = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            G.add_edge(u, v)

        return G, mol

    def relabel_graph_to_integers(self):
        mapping = {old: new for new, old in enumerate(self.graph.nodes())}
        G = nx.relabel_nodes(self.graph, mapping, copy=True)
        for new_idx, old_idx in mapping.items():
            G.nodes[new_idx]['element'] = self.graph.nodes[old_idx]['element']
            G.nodes[new_idx]['pos'] = self.graph.nodes[old_idx]['pos']
        return G

    def show_all_views(self):
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # RDKit
        img = Draw.MolToImage(self.mol, size=(300, 300), kekulize=True)
        axes[0].imshow(img)
        axes[0].set_title("RDKit Molecule")
        axes[0].axis("off")

        # NetworkX (element labels)
        pos = nx.get_node_attributes(self.graph, "pos")
        labels_element = nx.get_node_attributes(self.graph, "element")
        nx.draw(self.graph, pos, with_labels=True, labels=labels_element,
                node_color='lightblue', node_size=500, font_size=10, ax=axes[1])
        axes[1].set_title(self.compound_name + " (Atoms)")
        axes[1].axis("off")

        # NetworkX (index labels)
        labels_index = {i: i for i in self.graph.nodes()}
        nx.draw(self.graph, pos, with_labels=True, labels=labels_index,
                node_color='lightgreen', node_size=500, font_size=10, ax=axes[2])
        axes[2].set_title(self.compound_name + " (Indices)")
        axes[2].axis("off")

        plt.tight_layout()
        plt.show()

    def get_graph(self):
        return self.graph

    def get_smiles(self):
        return self.smiles

    def get_molecule(self):
        return self.mol

class BulkMolecularGraphExtractor:
    def __init__(self, compound_list, relabel=True, visualize=False):
        self.compound_list = compound_list
        self.relabel = relabel
        self.visualize = visualize
        self.data = []

        self.process_all()

    def process_all(self):
        for name in self.compound_list:
            try:
                extractor = MolecularGraphExtractor(name, relabel=self.relabel, visualize=self.visualize)
                self.data.append({
                    "name": name,
                    "graph": extractor.get_graph(),
                    "mol": extractor.get_molecule(),
                    "smiles": extractor.get_smiles()
                })
            except Exception as e:
                continue

    def get_all_graphs(self):
        return [d["graph"] for d in self.data]

    def to_dataframe(self):
        df = pd.DataFrame([{
            "Name": d["name"],
            "SMILES": d["smiles"],
            "NumVertices": d["graph"].number_of_nodes(),
            "NumEdges": d["graph"].number_of_edges()
        } for d in self.data])
        display(df)
        return df

    def apply_function(self, func):
        for entry in self.data:
            try:
                result = func(entry["graph"])
                if all(v is None for v in result.values()):
                    continue
                flat_result = {}
                for k, v in result.items():
                    if isinstance(v, dict):
                        for subk, subv in v.items():
                            flat_result[f"{k} [{subk}]"] = subv
                    else:
                        flat_result[k] = v
                df = pd.DataFrame(flat_result.items(), columns=["Property", "Value"])
                def format_latex(val):
                    if hasattr(val, 'latex'):
                        return f'${sp.latex(val)}$'
                    try:
                        return f'${sp.latex(sp.sympify(val))}$'
                    except:
                        return str(val)
                df["Value"] = df["Value"].apply(format_latex)
                display(HTML(f'<h3>{entry["name"]}</h3>'))
                display(HTML(df.to_html(escape=False, index=False)))
            except Exception:
                continue

In [272]:
class GraphPolynomials:
    def __init__(self, G):
        if not isinstance(G, nx.Graph):
            raise TypeError("Graph must be a networkx.Graph")
        self.G = G.copy()
        self.x = sympy.Symbol("x")
        self.y = sympy.Symbol("y")
        self.nodes = list(self.G.nodes)
        self.edges = list(self.G.edges)
        self.n = self.G.number_of_nodes()
        self.m = self.G.number_of_edges()
        self.u = sympy.Symbol("u")
        self.v = sympy.Symbol("v")
        self.p = sympy.Symbol("p")

    def chromatic_polynomial(self):
        def _compute(G):
            x = self.x
            if any(u == v for u, v in G.edges()):
                return 0
            if G.number_of_edges() == 0:
                return x ** G.number_of_nodes()
            u, v = list(G.edges())[0]
            G_del = G.copy()
            G_del.remove_edge(u, v)
            G_con = nx.contracted_nodes(G.copy(), u, v, self_loops=False)
            return _compute(G_del) - _compute(G_con)
        return sympy.simplify(_compute(self.G))

    def m_polynomial(self):
        degree_pairs = defaultdict(int)
        for u, v in self.edges:
            du, dv = self.G.degree[u], self.G.degree[v]
            i, j = sorted((du, dv))
            degree_pairs[(i, j)] += 1
        poly = sum(k * self.x**i * self.y**j for (i, j), k in degree_pairs.items())
        return sympy.simplify(poly)

    def nm_polynomial(self):
        degree_pairs = defaultdict(int)
        for u, v in self.edges:
            du, dv = self.G.degree[u], self.G.degree[v]
            # DO NOT sort. k ≤ l must preserve ordering from u to v
            degree_pairs[(du, dv)] += 1  # strictly (k, l)

        poly = sum(k * self.x**i * self.y**j for (i, j), k in degree_pairs.items())
        return sympy.simplify(poly)

    def independence_polynomial(self):
        alpha = len(nx.algorithms.approximation.maximum_independent_set(self.G))
        s_k = [0] * (alpha + 1)
        for k in range(alpha + 1):
            for subset in combinations(self.nodes, k):
                if all(not self.G.has_edge(u, v) for u, v in combinations(subset, 2)):
                    s_k[k] += 1
        poly = sum(s_k[k] * self.x**k for k in range(alpha + 1))
        return sympy.simplify(poly)

    def matching_polynomial(self):
        nu = len(nx.algorithms.maximal_matching(self.G))
        m_k = [0] * (nu + 1)
        for k in range(nu + 1):
            for edge_set in combinations(self.edges, k):
                nodes_used = set()
                if all(u not in nodes_used and v not in nodes_used and not nodes_used.update((u, v)) for u, v in edge_set):
                    m_k[k] += 1
        poly_expr = sum((-1)**k * m_k[k] * self.x**(self.n - 2*k) for k in range(nu + 1))
        return sympy.simplify(poly_expr)

    def domination_polynomial(self):
        counts = {}
        for k in range(1, self.n + 1):
            count = 0
            for subset in combinations(self.nodes, k):
                if is_dominating_set(self.G, subset):
                    count += 1
            if count > 0:
                counts[k] = count
        if not counts:
            return 0
        gamma = min(counts.keys())
        poly = sum(coeff * self.x**k for k, coeff in counts.items() if k >= gamma)
        return sympy.simplify(poly)

    def clique_polynomial(self):
        clique_counts = defaultdict(int)
        for clique in nx.enumerate_all_cliques(self.G):
            k = len(clique)
            if k >= 1:
                clique_counts[k] += 1
        poly = 1 + sum(count * self.x**k for k, count in clique_counts.items())
        return sympy.simplify(poly)

    def vertex_cover_polynomial(self):
        cover_counts = {}
        for k in range(1, self.n + 1):
            count = 0
            for subset in combinations(self.nodes, k):
                covered_edges = set()
                for u, v in self.edges:
                    if u in subset or v in subset:
                        covered_edges.add((u, v))
                if len(covered_edges) == self.m:
                    count += 1
            if count > 0:
                cover_counts[k] = count
        if not cover_counts:
            return 0
        tau = min(cover_counts.keys())
        poly = sum(count * self.x**k for k, count in cover_counts.items() if k >= tau)
        return sympy.simplify(poly)

    def edge_cover_polynomial(self):
        cover_counts = {}
        for k in range(1, self.m + 1):
            count = 0
            for edge_set in combinations(self.edges, k):
                covered_nodes = set()
                for u, v in edge_set:
                    covered_nodes.update([u, v])
                if covered_nodes == set(self.nodes):
                    count += 1
            if count > 0:
                cover_counts[k] = count
        if not cover_counts:
            return 0
        rho = min(cover_counts.keys())
        poly = sum(count * self.x**k for k, count in cover_counts.items() if k >= rho)
        return sympy.simplify(poly)
    
    def laplacian_polynomial(self):
        L = sympy.Matrix(nx.laplacian_matrix(self.G).toarray())
        return ((self.x * sympy.eye(self.n) - L).det()).expand()
    
    def adjacency_polynomial(self):
        A = sympy.Matrix(nx.adjacency_matrix(self.G).toarray())
        return ((self.x * sympy.eye(self.n) - A).det()).expand()
    
    def distance_polynomial(self):
        D = nx.floyd_warshall_numpy(self.G)
        poly = 0
        for i in range(self.n):
            for j in range(i + 1, self.n):
                if D[i, j] < float('inf'):
                    poly += self.x ** int(D[i, j])
        return sympy.simplify(poly)
    
    def rank_polynomial(self):
        rG = self.n - nx.number_connected_components(self.G)
        poly = 0
        for k in range(self.m + 1):
            for edge_subset in combinations(self.edges, k):
                subG = self.G.edge_subgraph(edge_subset).copy()
                rA = subG.number_of_nodes() - nx.number_connected_components(subG)
                x_term = self.x ** (rG - rA)
                y_term = self.y ** (k - rA)
                poly += x_term * y_term
        return sympy.simplify(poly)

    def tutte_polynomial(self):
        rG = self.n - nx.number_connected_components(self.G)
        T = 0
        for k in range(self.m + 1):
            for edge_subset in combinations(self.edges, k):
                subG = self.G.edge_subgraph(edge_subset).copy()
                rA = subG.number_of_nodes() - nx.number_connected_components(subG)
                x_term = (self.x - 1) ** (rG - rA)
                y_term = (self.y - 1) ** (k - rA)
                T += x_term * y_term
        return sympy.simplify(T)
    
    def reliability_polynomial(self):
        T = self.tutte_polynomial()
        R = (self.p ** (self.n - 1)) * T.subs({self.x: 1, self.y: 1 / self.p})
        return sympy.simplify(R)
    
    def dichromatic_polynomial(self):
        poly = 0
        for k in range(self.m + 1):
            for edge_subset in combinations(self.edges, k):
                subG = self.G.edge_subgraph(edge_subset).copy()
                num_components = nx.number_connected_components(subG)
                exponent = k - self.n + num_components
                poly += (self.u ** num_components) * (self.v ** exponent)
        return sympy.simplify(poly)


In [273]:
class GraphPartition:
    def __init__(self, G):
        if not isinstance(G, (nx.Graph, nx.MultiGraph)):
            raise TypeError("Graph must be a networkx.Graph or MultiGraph.")
        self.G = G.copy()
        self.nodes = list(G.nodes)
        self.edges = list(G.edges)
        self.degree_dict = dict(G.degree())

    def _show_dataframe(self, df, show):
        if show:
            display(df.style.hide(axis='index'))
            return None
        return df

    def degree_partition(self, show=True):
        degree_counts = Counter(self.degree_dict.values())
        df = pd.DataFrame(sorted(degree_counts.items()), columns=["Degree", "Count"])
        return self._show_dataframe(df, show)

    def distance_partition(self, show=True):
        lengths = dict(nx.floyd_warshall(self.G))
        dist_counter = defaultdict(int)

        for i, u in enumerate(self.nodes):
            for v in self.nodes[i + 1:]:
                d = lengths[u][v]
                if d != float('inf'):
                    dist_counter[d] += 1

        df = pd.DataFrame(sorted(dist_counter.items()), columns=["Distance", "Count"])
        return self._show_dataframe(df, show)

    def degree_pair_partition(self, show=True):
        degree_pairs = defaultdict(int)

        for u, v in self.edges:
            du, dv = self.degree_dict[u], self.degree_dict[v]
            i, j = sorted((du, dv))
            degree_pairs[(i, j)] += 1

        df = pd.DataFrame(
            sorted(((i, j, c) for (i, j), c in degree_pairs.items())),
            columns=["deg(u)", "deg(v)", "Count"]
        )
        return self._show_dataframe(df, show)

    def szeged_partition(self, show=True):
        lengths = dict(nx.all_pairs_shortest_path_length(self.G))
        rows = []

        for u, v in self.edges:
            nu = nv = 0
            for w in self.nodes:
                if lengths[w][u] < lengths[w][v]:
                    nu += 1
                elif lengths[w][v] < lengths[w][u]:
                    nv += 1
            rows.append({
                "Edge": (u, v),
                "n_u": nu,
                "n_v": nv,
                "Contribution": nu * nv
            })

        df = pd.DataFrame(rows)
        return self._show_dataframe(df, show)

In [274]:
def compute_all_polynomials(G):
    gp = GraphPolynomials(G)
    return {
        "M-Polynomial": gp.m_polynomial(),
        "NM-Polynomial": gp.nm_polynomial()
    }

def compute_partitions(G):
    gp2 = GraphPartition(G)
    return {
        "Degree Partition": gp2.degree_partition(),
        "Distance Partition": gp2.distance_partition()
    }

compounds = ["Cetirizine"]
bulk = BulkMolecularGraphExtractor(compounds, relabel=True, visualize=False)
bulk.apply_function(compute_all_polynomials)
bulk.apply_function(compute_partitions)

Property,Value
M-Polynomial,x*y**2*(3*x**2*y + 12*x*y + 11*x + 3*y)
NM-Polynomial,x**2*y*(3*x*y**2 + 7*x*y + 3*x + 5*y**2 + 11*y)


Degree,Count
1,3
2,17
3,7


Distance,Count
1.000000,29
2.000000,38
3.000000,38
4.000000,37
5.000000,40
6.000000,36
7.000000,30
8.000000,23
9.000000,17
10.000000,15
